Import Libraries

In [12]:
# General
import os                       # for file and path management
import sys                      # system-related utilities
import platform                 # to check system info

# Data Science, plotting
import numpy as np              # numerical computing
import pandas as pd             # data manipulation and tables
import matplotlib.pyplot as plt # plotting and visualization
import seaborn as sns           # advanced statistical plotting
import datetime                 # date/time utilities

# Bioinformatics
from Bio import SeqIO           # sequence input/output (from Biopython)
from Bio.Seq import Seq         # sequence object and manipulations
from Bio.SeqRecord import SeqRecord

# Statistics, modeling
import scipy                    # scientific computing
import statsmodels.api as sm    # statistics and models
import subprocess               # run shell commands if needed inside notebook


Testing Linux VM

In [13]:
print(platform.platform())

print("Python version:", sys.version)
print("Current working directory:", os.getcwd())
print("List files here:", os.listdir('.'))

!ls -l

Linux-3.10.0-1160.el7.x86_64-x86_64-with-glibc2.17
Python version: 3.10.13 (main, Sep 11 2023, 13:21:10) [GCC 11.2.0]
Current working directory: /media/sf_NGS-Analysis
List files here: ['.git', '.gitignore', '.ipynb_checkpoints', '.snakemake', 'data', 'logs', 'Notebook.ipynb', 'output', 'README.md', 'references', 'scripts', 'Snakefile', 'tmp']
total 56
drwxrwx---. 1 root vboxsf  4096 Aug  7 22:11 data
drwxrwx---. 1 root vboxsf     0 Aug  7 20:54 logs
-rwxrwx---. 1 root vboxsf 38231 Aug 10 10:44 Notebook.ipynb
drwxrwx---. 1 root vboxsf     0 Aug 10 11:11 output
-rwxrwx---. 1 root vboxsf  1988 Aug  8 12:22 README.md
drwxrwx---. 1 root vboxsf  4096 Aug 10 10:17 references
drwxrwx---. 1 root vboxsf     0 Aug  7 20:54 scripts
-rwxrwx---. 1 root vboxsf  2785 Aug  7 21:43 Snakefile
drwxrwx---. 1 root vboxsf     0 Aug  7 20:54 tmp


Make Output Directory and Check Tools

In [14]:
%%bash

set -u  # keep undefined var check, but don't exit on error

# Create output directory (replace sample01 with your sample name)
mkdir -p /output/F12345678910

# Check versions of tools
for t in bwa samtools bedtools bcftools gatk fastqc; do
  if command -v "$t" >/dev/null 2>&1; then
    echo "$t -> $($t --version 2>&1 | head -n1)"
  else
    echo "WARNING: $t not found in PATH"
  fi
done

bwa -> [main] unrecognized command '--version'
samtools -> samtools 1.5
bedtools -> bedtools v2.27.1
bcftools -> bcftools 1.6
gatk -> Using GATK jar /opt/gatk/gatk-package-4.6.1.0-local.jar
fastqc -> FastQC v0.12.1


Define Variables and File Paths

In [ ]:
# Large file locations
RAW_READS_DIR = os.getenv("RAW_READS_DIR", "/media/data_storage/reference/")
REF_DIR = os.getenv("REF_DIR", "/media/data_storage/reads/")

# Sample variables and file locations
SAMPLE="F12345678910"
PANEL="CompN"
R1 = f"{RAW_READS_DIR}/{SAMPLE}_R1.fastq.gz"
R2 = f"{RAW_READS_DIR}/{SAMPLE}_R2.fastq.gz"
REF = f"{REF_DIR}/Homo_sapiens.GRCh38.dna.primary_assembly.fa"
BED="/panels/{PANEL}.bed"
OUTDIR="/output/output_${SAMPLE}"
THREADS=8

# Export variables to environment
os.environ["SAMPLE"] = "F12345678910"
os.environ["RAW_READS_DIR"] = "/media/data_storage/reference"
os.environ["REF_DIR"] = "/media/data_storage/reads"
os.environ["PANEL"] = "CompN"
os.environ["THREADS"] = "8"

In [ ]:
%%bash

R1="${RAW_READS_DIR}/${SAMPLE}_R1.fastq.gz"
R2="${RAW_READS_DIR}/${SAMPLE}_R2.fastq.gz"
REF="${REF_DIR}/Homo_sapiens.GRCh38.dna.primary_assembly.fa"
BED="/media/data_storage/panels/${PANEL}.bed"
OUTDIR="/output/output_${SAMPLE}"

echo "SAMPLE: ${SAMPLE}"
echo "R1: ${R1}"
echo "R2: ${R2}"
echo "REF: ${REF}"
echo "BED: ${BED}"
echo "OUTDIR: ${OUTDIR}"
echo "THREADS: ${THREADS}"

SAMPLE: 
R1: 
R2: 
REF: 
BED: 
OUTDIR: 
THREADS: 


Check Input Files

In [21]:
%%bash

set -u
echo "Check FASTQ existence and first reads:"
#ls -lh "$R1" "$R2" || true
#zcat -n "$R1" | sed -n '1,4p' || true
#zcat -n "$R2" | sed -n '1,4p' || true

echo "Check BED header (first lines):"
head -n 5 "$BED" || true

echo "Check REF: show first header line and index status"
samtools faidx "$REF" >/dev/null 2>&1 || echo "fai missing (will create later)"
grep -m1 '^>' "$REF" || true

Check FASTQ existence and first reads:
Check BED header (first lines):


bash: line 9: BED: unbound variable


CalledProcessError: Command 'b'\nset -u\necho "Check FASTQ existence and first reads:"\n#ls -lh "$R1" "$R2" || true\n#zcat -n "$R1" | sed -n \'1,4p\' || true\n#zcat -n "$R2" | sed -n \'1,4p\' || true\n\necho "Check BED header (first lines):"\nhead -n 5 "$BED" || true\n\necho "Check REF: show first header line and index status"\nsamtools faidx "$REF" >/dev/null 2>&1 || echo "fai missing (will create later)"\ngrep -m1 \'^>\' "$REF" || true\n'' returned non-zero exit status 1.

Plots